In [1]:
pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.4/87.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11938 sha256=b18833236ee2f6e1e93c9968982418f8f3b5a1f2378d2400cf5565f8b5010a5b
  Stored in directory: /root/.cache/pip/wheels/af/f9/87/bf5b3d565c2a007b4dae9d8142dccc85a9f164e517062dd519
Successfully built stopit


In [2]:
import pandas as pd
from tpot import TPOTRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

#importing the requied libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
car = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Spring 2023 - AISC - Kuruma_CarPrice_Assignment.csv')
car


,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,201,-1,volvo 145e (sw),gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845.0
201,202,-1,volvo 144ea,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045.0
202,203,-1,volvo 244dl,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485.0
203,204,-1,volvo 246,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.40,23.0,106,4800,26,27,22470.0


In [4]:
#Splitting the Company name from Car Name column
Company_Name = car['CarName'].apply(lambda x : x.split(' ')[0])
car.insert(3,"CompanyName",Company_Name)
car.drop(['CarName'],axis=1,inplace=True)
car.head()

,car_ID,symboling,CompanyName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [5]:
car.isnull().sum()

car_ID              0
symboling           0
CompanyName         0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

In [6]:
car.CompanyName = car.CompanyName.str.lower()

def replace_name(a,b):
    car.CompanyName.replace(a,b,inplace=True)

replace_name('maxda','mazda')
replace_name('porcshce','porsche')
replace_name('toyouta','toyota')
replace_name('vokswagen','volkswagen')
replace_name('vw','volkswagen')

car.CompanyName.unique()

array(['alfa-romero', 'audi', 'bmw', 'chevrolet', 'dodge', 'honda',
       'isuzu', 'jaguar', 'mazda', 'buick', 'mercury', 'mitsubishi',
       'nissan', 'peugeot', 'plymouth', 'porsche', 'renault', 'saab',
       'subaru', 'toyota', 'volkswagen', 'volvo'], dtype=object)

In [7]:
#Checking for duplicate values in the data
car.loc[car.duplicated()]

,car_ID,symboling,CompanyName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price


In [8]:
#deriving the Fuel economy
car['fueleconomy'] = (0.55 * car['citympg']) + (0.45 * car['highwaympg'])

In [9]:
#Binning the car companies based on their average pricing.
car['price'] = car['price'].astype('int')
temp = car.copy()
table = temp.groupby(['CompanyName'])['price'].mean()
temp = temp.merge(table.reset_index(), how='left',on='CompanyName')
bins = [0,10000,20000,40000]
carsBin=['Budget','Medium','Highend']
car['carsrange'] = pd.cut(temp['price_y'],bins,right=False,labels=carsBin)

In [10]:
carsLr = car[['price', 'fueltype', 'aspiration','carbody', 'drivewheel','wheelbase',
                  'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower',
                    'fueleconomy', 'carlength','carwidth', 'carsrange']]
carsLr.head()

,price,fueltype,aspiration,carbody,drivewheel,wheelbase,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,fueleconomy,carlength,carwidth,carsrange
0,13495,gas,std,convertible,rwd,88.6,2548,dohc,four,130,3.47,111,23.70,168.8,64.1,Medium
1,16500,gas,std,convertible,rwd,88.6,2548,dohc,four,130,3.47,111,23.70,168.8,64.1,Medium
2,16500,gas,std,hatchback,rwd,94.5,2823,ohcv,six,152,2.68,154,22.15,171.2,65.5,Medium
3,13950,gas,std,sedan,fwd,99.8,2337,ohc,four,109,3.19,102,26.70,176.6,66.2,Medium
4,17450,gas,std,sedan,4wd,99.4,2824,ohc,five,136,3.19,115,19.80,176.6,66.4,Medium


In [11]:

# Define the map function
def dummies(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df


# Applied the function to the carsLr
carsLr = dummies('fueltype',carsLr)
carsLr = dummies('aspiration',carsLr)
carsLr = dummies('carbody',carsLr)
carsLr = dummies('drivewheel',carsLr)
carsLr = dummies('enginetype',carsLr)
carsLr = dummies('cylindernumber',carsLr)
carsLr = dummies('carsrange',carsLr)

In [12]:
from sklearn.model_selection import train_test_split

np.random.seed(0)
df_train, df_test = train_test_split(carsLr, train_size = 0.7, test_size = 0.3, random_state = 100)

In [13]:
#Dividing data into X and y variables
y_train = df_train.pop('price')
X_train = df_train


In [14]:
#Dividing into X and y
y_test = df_test.pop('price')
X_test = df_test


In [15]:
print(X_train)

     wheelbase  curbweight  enginesize  boreratio  horsepower  fueleconomy  \
122       93.7        2191          98       2.97          68        34.15   
125       94.5        2778         151       3.94         143        22.60   
166       94.5        2300          98       3.24         112        27.35   
1         88.6        2548         130       3.47         111        23.70   
199      104.3        3157         130       3.62         162        19.25   
..         ...         ...         ...        ...         ...          ...   
87        96.3        2403         110       3.17         116        26.15   
103      100.4        3060         181       3.43         152        21.70   
67       110.0        3515         183       3.58         123        23.35   
24        93.7        1967          90       2.97          68        34.15   
8        105.8        3086         131       3.13         140        18.35   

     carlength  carwidth  gas  turbo  ...  ohcv  rotor  five  f

In [16]:
print(y_train)

122     7609
125    22018
166     9538
1      16500
199    18950
       ...  
87      9279
103    13499
67     25552
24      6229
8      23875
Name: price, Length: 143, dtype: int64


In [24]:
print(X_train)

     wheelbase  curbweight  enginesize  boreratio  horsepower  fueleconomy  \
122       93.7        2191          98       2.97          68        34.15   
125       94.5        2778         151       3.94         143        22.60   
166       94.5        2300          98       3.24         112        27.35   
1         88.6        2548         130       3.47         111        23.70   
199      104.3        3157         130       3.62         162        19.25   
..         ...         ...         ...        ...         ...          ...   
87        96.3        2403         110       3.17         116        26.15   
103      100.4        3060         181       3.43         152        21.70   
67       110.0        3515         183       3.58         123        23.35   
24        93.7        1967          90       2.97          68        34.15   
8        105.8        3086         131       3.13         140        18.35   

     carlength  carwidth  gas  turbo  ...  ohcv  rotor  five  f

In [25]:
print(y_train)

122     7609
125    22018
166     9538
1      16500
199    18950
       ...  
87      9279
103    13499
67     25552
24      6229
8      23875
Name: price, Length: 143, dtype: int64


In [20]:
tpot = TPOTRegressor(generations=10, population_size=800, scoring='r2', verbosity=2)
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/8800 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9188343183853986

Generation 2 - Current best internal CV score: 0.9188343183853986

Generation 3 - Current best internal CV score: 0.9204953399204179

Generation 4 - Current best internal CV score: 0.9204953399204179

Generation 5 - Current best internal CV score: 0.9204953399204179

Generation 6 - Current best internal CV score: 0.9261906925982835

Generation 7 - Current best internal CV score: 0.9261906925982835

Generation 8 - Current best internal CV score: 0.9261906925982835

Generation 9 - Current best internal CV score: 0.9261906925982835

Generation 10 - Current best internal CV score: 0.9261906925982835

Best pipeline: RidgeCV(MinMaxScaler(DecisionTreeRegressor(input_matrix, max_depth=7, min_samples_leaf=1, min_samples_split=2)))


TPOTRegressor(generations=10, population_size=800, scoring='r2', verbosity=2)

In [22]:
tpot = TPOTRegressor(generations=10, population_size=100, scoring='r2', verbosity=2)
tpot.fit(X_test, y_test)

Optimization Progress:   0%|          | 0/1100 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.856691479570473

Generation 2 - Current best internal CV score: 0.8812945500108146

Generation 3 - Current best internal CV score: 0.8812945500108146

Generation 4 - Current best internal CV score: 0.8812945500108146

Generation 5 - Current best internal CV score: 0.8812945500108146

Generation 6 - Current best internal CV score: 0.8812945500108146

Generation 7 - Current best internal CV score: 0.8812945500108146

Generation 8 - Current best internal CV score: 0.8812945500108146

Generation 9 - Current best internal CV score: 0.8812945500108146

Generation 10 - Current best internal CV score: 0.8812945500108146

Best pipeline: LassoLarsCV(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), normalize=False)


TPOTRegressor(generations=10, scoring='r2', verbosity=2)

In [26]:
print(tpot.score(X_test, y_test))



0.9685238605723786


In [27]:
optimized_pipeline = tpot.fitted_pipeline_


In [28]:
tpot.export('tpot_kuruma_pipeline.py')

In [ ]:

#data= pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Spring 2023 - AISC - Kuruma_CarPrice_Assignment.csv")
#data